In [1]:
#import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
pd.set_option('display.max_columns', 200)
import re
import collections

In [2]:
#read in data from ?fao.csv? and display head for initial examination
csv_file = "../Resources/fao.csv"
fao_data_df = pd.read_csv(csv_file, encoding="ANSI")
fao_data_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [3]:
# insert an id column and rename column 'Item' to 'FAO_Category' for later merge
# https://stackoverflow.com/questions/38862293/how-to-add-incremental-numbers-to-a-new-column-using-pandas/38862389

fao_data_df.insert(0, 'ID', range(0, 0 + len(fao_data_df)))
fao_data_df = fao_data_df.rename(columns={'Item':'FAO_Category'})
fao_data_df

,ID,Area Abbreviation,Area Code,Area,Item Code,FAO_Category,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200
5,5,AFG,2,Afghanistan,2514,Maize and products,5142,Food,1000 tonnes,33.94,67.71,403.0,403.0,410.0,415.0,415.0,413.0,454.0,448.0,455.0,383.0,386.0,416.0,439.0,445.0,451.0,463.0,439.0,451.0,440.0,437.0,407.0,384.0,371.0,353.0,334.0,330.0,298.0,287.0,265.0,279.0,245.0,170.0,272.0,289.0,310.0,209.0,173.0,192.0,141.0,66.0,93.0,170.0,117.0,231.0,67.0,82.0,67.0,69.0,71.0,82.0,73.0,77,76
6,6,AFG,2,Afghanistan,2517,Millet and products,5142,Food,1000 tonnes,33.94,67.71,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,26.0,27.0,27.0,28.0,29.0,37.0,32.0,33.0,31.0,31.0,29.0,27.0,28.0,26.0,25.0,23.0,23.0,23.0,23.0,24.0,24.0,18.0,22.0,20.0,21.0,17.0,20.0,21.0,17.0,20.0,20.0,18.0,16.0,15.0,21.0,11.0,19.0,21.0,18.0,14.0,14.0,14,12
7,7,AFG,2,Afghanistan,2520,"Cereals, Other",5142,Food,1000 tonnes,33.94,67.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
8,8,AFG,2,Afghanistan,2531,Potatoes and products,5142,Food,1000 tonnes,33.94,67.71,111.0,97.0,103.0,110.0,113.0,117.0,128.0,130.0,134.0,125.0,147.0,138.0,143.0,160.0,169.0,324.0,176.0,225.0,232.0,240.0,247.0,248.0,242.0,235.0,226.0,217.0,196.0,198.0,184.0,205.0,203.0,210.0,210.0,211.0,212.0,213.0,214.0,214.0,217.0,219.0,215.0,217.0,347.0,276.0,294.0,294.0,260.0,242.0,250.0,192.0,169.0,196,230
9,9,AFG,2,Afghanistan,2536,Sugar cane,5521,Feed,1000 tonnes,33.94,67.71,45.0,45.0,45.0,45.0,31.0,14.0,19.0,30.0,34.0,15.0,0.0,0.0,28.0,32.0,20.0,28.0,24.0,24.0,34.0,61.0,50.0,43.0,38.0,46.0,23.0,25.0,3.

In [4]:
#create a list from a unique set of the 'Item' categories from initial dataset
# create a dataframe to be exported to csv
FAO_ctgry_df = pd.DataFrame({'FAO_Category': list(set(fao_data_df['FAO_Category']))})
FAO_ctgry_df.to_csv('./../FAO_ctgry_df.csv')
FAO_ctgry_df.head()

,FAO_Category
0,Grapefruit and products
1,Wine
2,Groundnut Oil
3,"Fats, Animals, Raw"
4,Sesame seed


Analyze categories and create column of normalized category 'bins' in excel

In [5]:
#read category datagrame with new column set index to normalized category
csv_file = "./../FAO_norm_ctgry_df.csv"
FAO_nrm_ctgry_df = pd.read_csv(csv_file)
#FAO_nrm_ctgry_df = FAO_nrm_ctgry_df.set_index('normalized_category')
FAO_nrm_ctgry_df.head()

,normalized_category,FAO_Category
0,Oil,"Oilcrops, Other"
1,Grain,"Pulses, Other and products"
2,Spice,Pepper
3,Grain,Cereals - Excluding Beer
4,Plant,Cassava and products


In [6]:
# merge to add normalized category to FAO
fao_norm_df = pd.merge(fao_data_df, FAO_nrm_ctgry_df, on='FAO_Category', how='left')
fao_norm_df

,ID,Area Abbreviation,Area Code,Area,Item Code,FAO_Category,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,normalized_category
0,0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895,Grain
1,1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422,Grain
2,2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360,Grain
3,3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89,Grain
4,4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200,Grain
5,5,AFG,2,Afghanistan,2514,Maize and products,5142,Food,1000 tonnes,33.94,67.71,403.0,403.0,410.0,415.0,415.0,413.0,454.0,448.0,455.0,383.0,386.0,416.0,439.0,445.0,451.0,463.0,439.0,451.0,440.0,437.0,407.0,384.0,371.0,353.0,334.0,330.0,298.0,287.0,265.0,279.0,245.0,170.0,272.0,289.0,310.0,209.0,173.0,192.0,141.0,66.0,93.0,170.0,117.0,231.0,67.0,82.0,67.0,69.0,71.0,82.0,73.0,77,76,Grain
6,6,AFG,2,Afghanistan,2517,Millet and products,5142,Food,1000 tonnes,33.94,67.71,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,26.0,27.0,27.0,28.0,29.0,37.0,32.0,33.0,31.0,31.0,29.0,27.0,28.0,26.0,25.0,23.0,23.0,23.0,23.0,24.0,24.0,18.0,22.0,20.0,21.0,17.0,20.0,21.0,17.0,20.0,20.0,18.0,16.0,15.0,21.0,11.0,19.0,21.0,18.0,14.0,14.0,14,12,Grain
7,7,AFG,2,Afghanistan,2520,"Cereals, Other",5142,Food,1000 tonnes,33.94,67.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,Alcohol
8,8,AFG,2,Afghanistan,2531,Potatoes and products,5142,Food,1000 tonnes,33.94,67.71,111.0,97.0,103.0,110.0,113.0,117.0,128.0,130.0,134.0,125.0,147.0,138.0,143.0,160.0,169.0,324.0,176.0,225.0,232.0,240.0,247.0,248.0,242.0,235.0,226.0,217.0,196.0,198.0,184.0,205.0,203.0,210.0,210.0,211.0,212.0,213.0,214.0,214.0,217.0,219.0,215.0,217.0,347.0,276.0,294.0,294.0,260.0,242.0,250.0,192.0,169.0,196,230,Plant
9,9,AFG,2,Afghanistan,2536,Sugar cane,5521,Feed,1000 tonnes,33.94,67.71,45.0,45.0,45.0,45.0,31.0,14.0,19.0,30.0,34.0,15.0,0.0,

In [7]:
#import second databaes low_memory=False option is way to show non-truncated columns

csv_file = "../Resources/en.openfoodfacts.org.products.tsv"
ofd_data_df = pd.read_csv(csv_file, sep='\t', low_memory=False)
ofd_data_df.head()


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, cor

In [8]:
# remove the columns with multirow text as initial cleanup to the massive open food database
mini_food_df = ofd_data_df[ofd_data_df.columns.difference(['ingredients_text', 'states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]


In [9]:
# OFD with ingredients for potential categorizations ordered by column with fewest NaN
food_ing_df = ofd_data_df[ofd_data_df.columns.difference(['states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]
food_ing_df = food_ing_df[['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
#food_ing_df = food_df.dropna(axis=0, subset=['categories_en'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['energy_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['proteins_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['carbohydrates_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['saturated-fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fiber_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['sugars_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['ingredients_text'])
food_ing_df.columns

Index(['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime',
       'code', 'countries_en', 'countries_tags', 'countries', 'product_name',
       'brands', 'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 

In [10]:
# create dataframe from ingredients list and pare down to main ingredient (first) and remove non-english characters using regex
# write pared ingredient list to csv
OFD_ing_df = pd.DataFrame({'OFD_Ingredients': list(set(food_ing_df['ingredients_text']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ing_df['Main_Ingredient'] = OFD_ing_df['OFD_Ingredients'].str.split(',').str[0]
OFD_ing_df = OFD_ing_df[~OFD_ing_df['Main_Ingredient'].astype(str).str.contains(r'[^A-Za-z]', regex=True)]
#OFD_ing_df
OFD_ing_df['Main_Ingredient'].to_csv('./../OFD_ing_df.csv')
OFD_Ing_List = list(OFD_ing_df['Main_Ingredient'])

In [11]:
# define function to determine percentage of missing values in each column
# https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe

def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [12]:
# run reduced dataframe through function to ascertain which columns contain lots of empty values
# sort values in ascending order so cleanest columns are at the top
miss_df = missing_values_table(mini_food_df)
miss_df = miss_df.sort_values('% of Total Values', ascending=True)
miss_df.head() 

Your selected dataframe has 156 columns.
There are 154 columns that have missing values.


,Missing Values,% of Total Values
created_t,3,0.0
url,26,0.0
creator,3,0.0
created_datetime,10,0.0
code,26,0.0


In [13]:
# limit missing data df to columns with at least 90% of total values
# print index (list of columns) to screen for copy/paste
miss_df = miss_df.loc[miss_df['% of Total Values'] < 90, :]
miss_df.index

Index(['created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_place

In [14]:
# create new dataframe with only desired columns in desired order with the good old copy-paste

ofd_rel_df = ofd_data_df[['created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
ofd_rel_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
0,1474103866,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2016-09-17T09:17:46Z,0000000003087,France,en:france,en:FR,Farine de blé noir,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004530,United States,en:united-states,US,Banana Chips Sweetened (Whole),NaN,NaN,2243.0,3.57,0.00000,0.000,0.0,0.0,28.57,14.29,64.29,28.57,14.0,d,14.0,3.6,28 g (1 ONZ),NaN,0.018,0.0,0.000,0.0214,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1489069957,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T14:32:37Z,0000000004559,United States,en:united-states,US,Peanuts,Torn & Glasser,torn-glasser,1941.0,17.86,0.63500,0.250,0.0,0.0,17.86,17.86,60.71,0.00,0.0,b,0.0,7.1,28 g (0.25 cup),NaN,0.000,0.0,0.071,0.0000,0.00129,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1489055731,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:35:31Z,0000000016087,United States,en:united-states,US,Organic Salted Nut Mix,Grizzlies,grizzlies,2540.0,17.86,1.22428,0.482,0.0,0.0,57.14,3.57,17.86,5.36,12.0,d,12.0,7.1,28 g (0.25 cup),NaN,NaN,NaN,0.143,NaN,0.00514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1489055653,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09T10:34:13Z,0000000016094,United States,en:united-states,US,Organic Polenta,Bob's Red Mill,bob-s-red-mill,1552.0,8.57,NaN,NaN,0.0,0.0,1.43,NaN,77.14,NaN,NaN,NaN,NaN,5.7,35 g (0.25 cup),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# remove all rows with NaN values in relevant columns
relevant_cats= ['categories_en', 'energy_100g', 'proteins_100g', 'carbohydrates_100g', 
                'fat_100g', 'saturated-fat_100g', 'fiber_100g', 'sugars_100g']
clean_df = ofd_rel_df.dropna(axis=0, subset=['categories_en'])
clean_df = clean_df.dropna(axis=0, subset=['energy_100g'])
clean_df = clean_df.dropna(axis=0, subset=['proteins_100g'])
clean_df = clean_df.dropna(axis=0, subset=['carbohydrates_100g'])
clean_df = clean_df.dropna(axis=0, subset=['saturated-fat_100g'])
clean_df = clean_df.dropna(axis=0, subset=['fiber_100g'])
clean_df = clean_df.dropna(axis=0, subset=['fat_100g'])
clean_df = clean_df.dropna(axis=0, subset=['sugars_100g'])
clean_df.head()


,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens
176,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.1600,0.8500,0.0,0.0,12.00,0.000,23.00,7.000,6.0,c,6.0,2.000,100 g,NaN,0.05600,NaN,0.0000,0.000000,0.000000,0.000000,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN
182,1484501528,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15T17:32:08Z,0000001938067,Canada,en:canada,Canada,Chaussons tressés aux pommes,Kirkland Signature,kirkland-signature,1090.0,3.33,0.6470,0.2550,0.0,0.0,10.70,24.700,38.70,2.000,9.0,c,9.0,2.000,150 g / 1 chausson,"en:e202,en:e211,en:e330,en:e509,en:e920",0.00533,0.667,0.0133,0.001600,0.000480,0.000000,Biscuits and cakes,Sugary snacks,1.200 kg,"en:sugary-snacks,en:biscuits-and-cakes,en:past...",Sugary snacks,en:sugary-snacks,"Snacks sucrés,Biscuits et gâteaux,Pâtisseries","Sugary snacks,Biscuits and cakes,Pastries",frais,Frais,NaN,NaN,Brossard Québec,NaN,NaN,NaN,Pâtisserie,Costco,Brossard Québec,brossard-quebec,NaN
183,1488464896,http://world-en.openfoodfacts.org/product/0000...,b7,2017-03-02T14:28:16Z,0000004302544,Canada,en:canada,Canada,Pain Burger Artisan,Kirkland Signature,kirkland-signature,1160.0,10.00,1.5200,0.6000,0.0,0.0,1.11,1.110,53.30,0.333,1.0,b,1.0,2.220,90 g / 1 pain,NaN,0.00000,0.000,0.0222,0.000000,0.006000,0.000000,unknown,unknown,1.008 kg / 12 pain,fr:boulange,fr:Boulange,fr:boulange,Boulange,fr:Boulange,"frais,plastique","Frais,plastique",NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Pain,Costco,"Brossard,Québec","brossard,quebec",NaN
185,1420150193,http://world-en.openfoodfacts.org/product/0000...,teolemon,2015-01-01T22:09:53Z,0000007020254,France,en:france,France,Root Beer,A&W,a-w,215.0,0.00,0.0616,0.0242,0.0,0.0,0.00,13.600,14.20,0.000,18.0,e,3.0,0.000,33 cl,"en:e150,en:e211,en:e999",NaN,NaN,NaN,NaN,NaN,NaN,Sweetened beverages,Beverages,33 cl e,"en:beverages,en:carbonated-drinks,en:sodas,en:...",Beverages,en:beverages,"Boissons,Boissons gazeuses,Sodas,Boissons sucr...","Beverages,Carbonated drinks,Sodas,Sugared beve...","canette,metal","Canette,Métal",http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,France,"Kosher,fr:Contient-des-ogm","Kascher,Contient des OGM","en:kosher,fr:contient-des-ogm",NaN,NaN,États-Unis,etats-unis,NaN
187,1491943307,http://world-en.openfoodfacts.org/product/0000...,b7,2017-04-11T20:41:47Z,0000008237798,Canada,en:canada,Canada,Quiche Lorraine,Kirkland Signature,kirkland-signature,478.0,5.36,0.4990,0.1960,0.0,0.0,6.79,0.714,7.86,2.860,2.0,b,2.0,0.357,280 g / 1/5 de la Quiche,"en:e235,en:e316,en:e407",0.01070,0.000,0.0286,0.000429,0.000643,0.000011,Pizza pies and quiche,Composite foods,1 400 kg,"en:meals,en:pizzas-pies-and-quiches,en:quiches...",Meals,en:meals,Quiches lorraines,"Meals,Pizzas pies and quiches,Quiches,fr:Quich...",frai,Frai,NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Quiche lorraine,Cost

In [16]:
#create category dataframe from relevant clean_df and append a boolean column stating if category is in english
# 3,640 categoryies in original df, 1,287 after removing non-english; 1463 categories in clean_df, 483 after non-english filter
OFD_ctgry_df = pd.DataFrame({'main_category': list(set(clean_df['main_category']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['main_category'].str.match(r'en:')
OFD_ctgry_df.head()

,main_category,en_boolean
0,es:natas-para-montar,False
1,en:suppenwurfel,True
2,fr:panini-rolls,False
3,fr:galettes-vegetales,False
4,fr:celeris-remoulade,False


In [17]:
# insert an id column
# https://stackoverflow.com/questions/38862293/how-to-add-incremental-numbers-to-a-new-column-using-pandas/38862389

OFD_ctgry_df.insert(0, 'ID', range(0, 0 + len(OFD_ctgry_df)))

OFD_ctgry_df.head()

,ID,main_category,en_boolean
0,0,es:natas-para-montar,False
1,1,en:suppenwurfel,True
2,2,fr:panini-rolls,False
3,3,fr:galettes-vegetales,False
4,4,fr:celeris-remoulade,False


In [18]:
# reduce dataframe to only english (supposedly, lol) categories and strip the prefix
# make list from column
OFD_en_ctgry_df = OFD_ctgry_df.loc[OFD_ctgry_df['en_boolean'] == True, :].reset_index().drop(columns=['index', 'en_boolean'])
OFD_en_ctgry_df['main_category'] = OFD_en_ctgry_df['main_category'].str.slice(3)
OFD_en_ctgry_df = OFD_en_ctgry_df.rename(columns={'main_category':'OFD_Category'})
OFD_en_ctgry_df.to_csv('./../OFD_en_ctgry_df.csv')

#OFD_Categories = list(OFD_en_ctgry_df['OFD_Category'])
#OFD_Categories

OFD_en_ctgry_df.head()

,ID,OFD_Category
0,1,suppenwurfel
1,6,pains-au-chocolat
2,8,energy-balls
3,11,penne
4,12,haferflocken


In [19]:
#read category datagrame with new column and option to set index to normalized category
csv_file = "./../OFD_en_norm_ctgry_df.csv"
OFD_nrm_ctgry_df = pd.read_csv(csv_file)
#OFD_nrm_ctgry_df = OFD_nrm_ctgry_df.set_index('normalized_category')
OFD_nrm_ctgry_df.head()


,normalized_category,OFD_Category
0,Dry_Goods,psyllium-husk
1,Meat,chicken-thighs
2,Dry_Goods,snacks
3,Dry_Goods,salted-snack
4,Meat,chorizos


In [20]:
# compare original (no reset index) df to compare the two category lists which will become primary keys to links the databases
csv_file = "./../OFD_en_norm_ctgry_df.csv"
OFD_nrm_ctgry_df = pd.read_csv(csv_file)
ofd_nrm_ctgry = list(OFD_nrm_ctgry_df['normalized_category'])
set(ofd_nrm_ctgry)


{'Coconut',
 'Dairy',
 'Dry_Goods',
 'Fruit',
 'Grain',
 'Meat',
 'Oil',
 'Other',
 'Palm',
 'Plant',
 'Seafood',
 'Seed',
 'Soy',
 'Spice',
 'Sweet',
 'Vegetable'}

In [21]:
# compare original (no reset index) df to compare the two category lists which will become primary keys to links the databases
csv_file = "./../FAO_norm_ctgry_df.csv"
FAO_nrm_ctgry_df = pd.read_csv(csv_file)
fao_nrm_ctgry = list(FAO_nrm_ctgry_df['normalized_category'])
set(fao_nrm_ctgry)

{'Alcohol',
 'Coconut',
 'Dairy',
 'Dry_Goods',
 'Fruit',
 'Grain',
 'Infant',
 'Meat',
 'Oil',
 'Other',
 'Palm',
 'Plant',
 'Seafood',
 'Seed',
 'Soy',
 'Spice',
 'Sweet',
 'Vegetable'}

In [22]:
# merge to get ID
ofd_norm1_df = pd.merge(OFD_en_ctgry_df, OFD_nrm_ctgry_df, on='OFD_Category', how='left')
ofd_norm1_df.head()

,ID,OFD_Category,normalized_category
0,1,suppenwurfel,Other
1,6,pains-au-chocolat,Sweet
2,8,energy-balls,Other
3,11,penne,Grain
4,12,haferflocken,Grain


In [23]:
# merge to get original category column
ofd_norm2_df = pd.merge(OFD_ctgry_df, ofd_norm1_df, on='ID', how='left')
ofd_norm2_df.head()

,ID,main_category,en_boolean,OFD_Category,normalized_category
0,0,es:natas-para-montar,False,NaN,NaN
1,1,en:suppenwurfel,True,suppenwurfel,Other
2,2,fr:panini-rolls,False,NaN,NaN
3,3,fr:galettes-vegetales,False,NaN,NaN
4,4,fr:celeris-remoulade,False,NaN,NaN


In [24]:
ofd_norm_df = ofd_norm2_df.dropna(axis=0, subset=['OFD_Category'])
ofd_norm_df.head()

,ID,main_category,en_boolean,OFD_Category,normalized_category
1,1,en:suppenwurfel,True,suppenwurfel,Other
6,6,en:pains-au-chocolat,True,pains-au-chocolat,Sweet
8,8,en:energy-balls,True,energy-balls,Other
11,11,en:penne,True,penne,Grain
12,12,en:haferflocken,True,haferflocken,Grain


In [25]:
# remove rows with NaN values and drop en_boolean column
ofd_clean_norm_df = pd.merge(clean_df, ofd_norm_df, on='main_category', how='left')
ofd_clean_norm_df = ofd_clean_norm_df.dropna(axis=0, subset=['normalized_category']).drop(columns='en_boolean')

ofd_clean_norm_df.head()

,created_t,url,creator,created_datetime,code,countries_en,countries_tags,countries,product_name,brands,brands_tags,energy_100g,proteins_100g,salt_100g,sodium_100g,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,fat_100g,sugars_100g,carbohydrates_100g,saturated-fat_100g,nutrition-score-fr_100g,nutrition_grade_fr,nutrition-score-uk_100g,fiber_100g,serving_size,additives_tags,cholesterol_100g,trans-fat_100g,calcium_100g,vitamin-c_100g,iron_100g,vitamin-a_100g,pnns_groups_2,pnns_groups_1,quantity,categories_tags,main_category_en,main_category,categories,categories_en,packaging_tags,packaging,image_url,image_small_url,purchase_places,labels_en,labels,labels_tags,generic_name,stores,manufacturing_places,manufacturing_places_tags,allergens,ID,OFD_Category,normalized_category
0,1484605978,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16T22:32:58Z,0000000290616,Canada,en:canada,Canada,Salade Cesar,Kirkland Signature,kirkland-signature,1210.0,22.00,2.1600,0.85000,0.0,0.0,12.00,0.000,23.00,7.00,6.0,c,6.0,2.000,100 g,NaN,0.05600,NaN,0.0000,0.000000,0.000000,0.000000,Vegetables,Fruits and vegetables,0.980 kg,"en:plant-based-foods-and-beverages,en:plant-ba...",Plant-based foods and beverages,en:plant-based-foods-and-beverages,Légumes-feuilles,"Plant-based foods and beverages,Plant-based fo...",frais,Frais,NaN,NaN,NaN,NaN,NaN,NaN,Laitue en salade,Costco,Brossard Québec,brossard-quebec,NaN,860.0,plant-based-foods-and-beverages,Other
1,1484501528,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15T17:32:08Z,0000001938067,Canada,en:canada,Canada,Chaussons tressés aux pommes,Kirkland Signature,kirkland-signature,1090.0,3.33,0.6470,0.25500,0.0,0.0,10.70,24.700,38.70,2.00,9.0,c,9.0,2.000,150 g / 1 chausson,"en:e202,en:e211,en:e330,en:e509,en:e920",0.00533,0.667,0.0133,0.001600,0.000480,0.000000,Biscuits and cakes,Sugary snacks,1.200 kg,"en:sugary-snacks,en:biscuits-and-cakes,en:past...",Sugary snacks,en:sugary-snacks,"Snacks sucrés,Biscuits et gâteaux,Pâtisseries","Sugary snacks,Biscuits and cakes,Pastries",frais,Frais,NaN,NaN,Brossard Québec,NaN,NaN,NaN,Pâtisserie,Costco,Brossard Québec,brossard-quebec,NaN,834.0,sugary-snacks,Sweet
3,1420150193,http://world-en.openfoodfacts.org/product/0000...,teolemon,2015-01-01T22:09:53Z,0000007020254,France,en:france,France,Root Beer,A&W,a-w,215.0,0.00,0.0616,0.02420,0.0,0.0,0.00,13.600,14.20,0.00,18.0,e,3.0,0.000,33 cl,"en:e150,en:e211,en:e999",NaN,NaN,NaN,NaN,NaN,NaN,Sweetened beverages,Beverages,33 cl e,"en:beverages,en:carbonated-drinks,en:sodas,en:...",Beverages,en:beverages,"Boissons,Boissons gazeuses,Sodas,Boissons sucr...","Beverages,Carbonated drinks,Sodas,Sugared beve...","canette,metal","Canette,Métal",http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,France,"Kosher,fr:Contient-des-ogm","Kascher,Contient des OGM","en:kosher,fr:contient-des-ogm",NaN,NaN,États-Unis,etats-unis,NaN,369.0,beverages,Other
4,1491943307,http://world-en.openfoodfacts.org/product/0000...,b7,2017-04-11T20:41:47Z,0000008237798,Canada,en:canada,Canada,Quiche Lorraine,Kirkland Signature,kirkland-signature,478.0,5.36,0.4990,0.19600,0.0,0.0,6.79,0.714,7.86,2.86,2.0,b,2.0,0.357,280 g / 1/5 de la Quiche,"en:e235,en:e316,en:e407",0.01070,0.000,0.0286,0.000429,0.000643,0.000011,Pizza pies and quiche,Composite foods,1 400 kg,"en:meals,en:pizzas-pies-and-quiches,en:quiches...",Meals,en:meals,Quiches lorraines,"Meals,Pizzas pies and quiches,Quiches,fr:Quich...",frai,Frai,NaN,NaN,"Brossard,Québec",NaN,NaN,NaN,Quiche lorraine,Costco,"Brossard,Québec","brossard,quebec",NaN,282.0,meals,Other
5,1418732959,http://world-en.openfoodfacts.org/product/0000...,tacinte,2014-12-16T12:29:19Z,0000027533024,United Kingdom,en:united-kingdom,United Kingdom,Luxury Christmas Pudding,"Asda,Asda Extra Special","asda,asda-extra-special",1284.0,3.90,0.2000,0.07874,0.0,0.0,7.00,44.000,54.20,3.60,10.0,c,10.0,4.700,113.5g,"en:e163,en:e170,en:e330,en:e375",NaN,NaN,NaN,Na

In [26]:
# memory error when trying to merge the dfs
# https://stackoverflow.com/questions/47386405/memoryerror-when-i-merge-two-pandas-data-frames
#fao_ofd_df = pd.merge(ofd_clean_norm_df, fao_norm_df, on='normalized_category', how='left')
#fao_ofd_df

In [27]:
# save final df to csv
#fao_ofd_df.to_csv('./../fao_ofd_df.csv')

A few groupings of the original dataframes

In [28]:
# group FAO data by 'Area' (country) and 'Item' (category) 

cntry_item_grp = fao_data_df.groupby(['Area', 'FAO_Category']).sum()
cntry_item_grp.head()

ID  Area Code  Item Code  Element Code  \
Area        FAO_Category                                                  
Afghanistan Alcoholic Beverages  74          2       2924          5142   
            Animal fats          77          2       2946          5142   
            Apples and products  34          2       2617          5142   
            Bananas              33          2       2615          5142   
            Barley and products   5          4       5026         10663   

                                 latitude  longitude  Y1961  Y1962  Y1963  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages     33.94      67.71    0.0    0.0    0.0   
            Animal fats             33.94      67.71   20.0   20.0   21.0   
            Apples and products     33.94      67.71   14.0   14.0   14.0   
            Bananas                 33.94      67.71    0.0    0.0    0.0   
            Barley and products     67.88     135.42  313.0  313.0  313.0   

                                 Y1964  Y1965  Y1966  Y1967  Y1968  Y1969  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           22.0   24.0   25.0   27.0   29.0   29.0   
            Apples and products   17.0   19.0   22.0   24.0   25.0   26.0   
            Bananas                0.0    0.0    0.0    0.0    0.0    0.0   
            Barley and products  314.0  314.0  312.0  296.0  299.0  303.0   

                                 Y1970  Y1971  Y1972  Y1973  Y1974  Y1975  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           27.0   26.0   25.0   27.0   28.0   29.0   
            Apples and products   24.0   20.0   20.0   22.0   21.0   21.0   
            Bananas                0.0    0.0    0.0    0.0    0.0    0.0   
            Barley and products  308.0  294.0  289.0  297.0  316.0  321.0   

                                 Y1976  Y1977  Y1978  Y1979  Y1980  Y1981  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           31.0   31.0   31.0   31.0   31.0   32.0   
            Apples and products   21.0   13.0   18.0   19.0   17.0   18.0   
            Bananas                0.0    0.0    1.0    1.0    1.0    9.0   
            Barley and products  335.0  245.0  268.0  262.0  266.0  249.0   

                                 Y1982  Y1983  Y1984  Y1985  Y1986  Y1987  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           32.0   32.0   30.0   26.0   20.0   22.0   
            Apples and products   16.0   15.0   13.0   15.0   16.0   11.0   
            Bananas               11.0    4.0    3.0    4.0    5.0    9.0   
            Barley and products  229.0  220.0  211.0  199.0  191.0  191.0   

                                 Y1988  Y1989  Y1990  Y1991  Y1992  Y1993  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           23.0   24.0   29.0   30.0   30.0   33.0   
            Apples and products   14.0   16.0   16.0   15.0   15.0   15.0   
            Bananas                6.0    0.0    0.0    1.0    4.0    3.0   
            Barley and products  195.0  191.0  178.0  175.0  160.0  205.0   

                                 Y1994  Y1995  Y1996  Y1997  Y1998  Y1999  \
Area        FAO_Category                                                    
Afghanistan Alcoholic Beverages    0.0    0.0    0.0    0.0    0.0    0.0   
            Animal fats           37

In [29]:
# drop NaN values, group by count of categories_en where count of products > 1, descending 
category_df = ofd_rel_df.dropna(axis=0, subset=['categories_en'])
grp_ctgry = category_df.groupby(['categories_en'])
grp_cnt = grp_ctgry.count().sort_values('created_t', ascending=False)
grp_cnt_df = pd.DataFrame(grp_cnt['created_t'])
grp_cnt_df = grp_cnt_df.rename(columns={'created_t': 'count'})
grp_cnt_df = grp_cnt_df.loc[grp_cnt_df['count'] > 1, :]
grp_cnt_df

,count
categories_en,
"Beverages,Non-sugared beverages",2345
"Sugary snacks,Biscuits and cakes,Biscuits",893
Fats,732
"Sugary snacks,Chocolates,Dark chocolates",678
"Sugary snacks,Confectioneries,Candies",587
"Groceries,Sauces,Tomato sauces,Ketchup",561
"Salty snacks,Appetizers,Crackers",534
"Dairies,Milks",504
"Sugary snacks,Biscuits and cakes,Biscuits,Chocolate biscuits",498


In [30]:
# drop NaN values, group b brands where count of products > 300, descending 

brand_df = ofd_rel_df.dropna(axis=0, subset=['brands'])
grp_brnd = brand_df.groupby(['brands'])
brnd_cnt = grp_brnd.count().sort_values('created_t', ascending=False)
brnd_cnt_df = pd.DataFrame(brnd_cnt['created_t'])
brnd_cnt_df = brnd_cnt_df.rename(columns={'created_t': 'count'})
brnd_cnt_df = brnd_cnt_df.loc[brnd_cnt_df['count'] > 300, :]
brnd_cnt_df

,count
brands,
Carrefour,3670
Auchan,3217
U,2608
Casino,2091
Leader Price,2015
Meijer,1995
Kroger,1662
Cora,1374
Ahold,1370


In [31]:
# drop NaN values, group by country where count of products > 300, descending 

country_df = ofd_rel_df.dropna(axis=0, subset=['countries_en'])
grp_cntry = country_df.groupby(['countries_en'])
cntry_cnt = grp_cntry.count().sort_values('created_t', ascending=False)
cntry_cnt_df = pd.DataFrame(cntry_cnt['created_t'])
cntry_cnt_df = cntry_cnt_df.rename(columns={'created_t': 'count'})
cntry_cnt_df = cntry_cnt_df.loc[cntry_cnt_df['count'] > 1, :]
cntry_cnt_df

,count
countries_en,
United States,173159
France,123961
Switzerland,14932
Germany,8470
Spain,5529
United Kingdom,5092
Belgium,2936
Australia,2201
Russia,1582


Attempts to create a dictionary of category: sub-categories to programatically classify either the ingredients or categories

In [32]:
#print statements to test output of itterows functions on naked row element (nothing in [])
#create list of tuples as key, value pairs to map a dictionary of categories
fao_cat_tuples= []
for index, row in FAO_nrm_ctgry_df.iterrows():
    #print(index)
    #print("--------------------------------")
    #print(row['FAO_Category'])
    #print("--------------------------------")
    fao_cat_tuples.append(tuple([index, row['FAO_Category']]))
fao_cat_tuples

[(0, 'Oilcrops, Other'),
 (1, 'Pulses, Other and products'),
 (2, 'Pepper'),
 (3, 'Cereals - Excluding Beer'),
 (4, 'Cassava and products'),
 (5, 'Dates'),
 (6, 'Cephalopods'),
 (7, 'Maize Germ Oil'),
 (8, 'Nuts and products'),
 (9, 'Cereals, Other'),
 (10, 'Bananas'),
 (11, 'Cocoa Beans and products'),
 (12, 'Poultry Meat'),
 (13, 'Mutton & Goat Meat'),
 (14, 'Pigmeat'),
 (15, 'Cottonseed'),
 (16, 'Bovine Meat'),
 (17, 'Stimulants'),
 (18, 'Offals'),
 (19, 'Pineapples and products'),
 (20, 'Fish, Body Oil'),
 (21, 'Cream'),
 (22, 'Rice (Milled Equivalent)'),
 (23, 'Aquatic Products, Other'),
 (24, 'Crustaceans'),
 (25, 'Offals, Edible'),
 (26, 'Cottonseed Oil'),
 (27, 'Coconuts - Incl Copra'),
 (28, 'Ricebran Oil'),
 (29, 'Freshwater Fish'),
 (30, 'Butter, Ghee'),
 (31, 'Meat, Other'),
 (32, 'Grapes and products (excl wine)'),
 (33, 'Sugar cane'),
 (34, 'Eggs'),
 (35, 'Infant food'),
 (36, 'Roots, Other'),
 (37, 'Animal fats'),
 (38, 'Demersal Fish'),
 (39, 'Treenuts'),
 (40, 'Oilcrop

In [33]:
#create dictionary mapping FAO_Categries to their normalized counterparts
# https://stackoverflow.com/questions/1747817/create-a-dictionary-with-list-comprehension-in-python

fao_cat_dict = {}
for (key, value) in fao_cat_tuples:
    if key not in fao_cat_dict:
        fao_cat_dict[key] = []
    fao_cat_dict[key].append(value)
fao_cat_dict

{0: ['Oilcrops, Other'],
 1: ['Pulses, Other and products'],
 2: ['Pepper'],
 3: ['Cereals - Excluding Beer'],
 4: ['Cassava and products'],
 5: ['Dates'],
 6: ['Cephalopods'],
 7: ['Maize Germ Oil'],
 8: ['Nuts and products'],
 9: ['Cereals, Other'],
 10: ['Bananas'],
 11: ['Cocoa Beans and products'],
 12: ['Poultry Meat'],
 13: ['Mutton & Goat Meat'],
 14: ['Pigmeat'],
 15: ['Cottonseed'],
 16: ['Bovine Meat'],
 17: ['Stimulants'],
 18: ['Offals'],
 19: ['Pineapples and products'],
 20: ['Fish, Body Oil'],
 21: ['Cream'],
 22: ['Rice (Milled Equivalent)'],
 23: ['Aquatic Products, Other'],
 24: ['Crustaceans'],
 25: ['Offals, Edible'],
 26: ['Cottonseed Oil'],
 27: ['Coconuts - Incl Copra'],
 28: ['Ricebran Oil'],
 29: ['Freshwater Fish'],
 30: ['Butter, Ghee'],
 31: ['Meat, Other'],
 32: ['Grapes and products (excl wine)'],
 33: ['Sugar cane'],
 34: ['Eggs'],
 35: ['Infant food'],
 36: ['Roots, Other'],
 37: ['Animal fats'],
 38: ['Demersal Fish'],
 39: ['Treenuts'],
 40: ['Oilcrop

In [34]:
# dictionary method 2
fao_cat_dict = {}
for (key, value) in fao_cat_tuples:
    fao_cat_dict.setdefault(key, []).append(value)
fao_cat_dict

{0: ['Oilcrops, Other'],
 1: ['Pulses, Other and products'],
 2: ['Pepper'],
 3: ['Cereals - Excluding Beer'],
 4: ['Cassava and products'],
 5: ['Dates'],
 6: ['Cephalopods'],
 7: ['Maize Germ Oil'],
 8: ['Nuts and products'],
 9: ['Cereals, Other'],
 10: ['Bananas'],
 11: ['Cocoa Beans and products'],
 12: ['Poultry Meat'],
 13: ['Mutton & Goat Meat'],
 14: ['Pigmeat'],
 15: ['Cottonseed'],
 16: ['Bovine Meat'],
 17: ['Stimulants'],
 18: ['Offals'],
 19: ['Pineapples and products'],
 20: ['Fish, Body Oil'],
 21: ['Cream'],
 22: ['Rice (Milled Equivalent)'],
 23: ['Aquatic Products, Other'],
 24: ['Crustaceans'],
 25: ['Offals, Edible'],
 26: ['Cottonseed Oil'],
 27: ['Coconuts - Incl Copra'],
 28: ['Ricebran Oil'],
 29: ['Freshwater Fish'],
 30: ['Butter, Ghee'],
 31: ['Meat, Other'],
 32: ['Grapes and products (excl wine)'],
 33: ['Sugar cane'],
 34: ['Eggs'],
 35: ['Infant food'],
 36: ['Roots, Other'],
 37: ['Animal fats'],
 38: ['Demersal Fish'],
 39: ['Treenuts'],
 40: ['Oilcrop

In [35]:
# method 3
# https://stackoverflow.com/questions/1747817/create-a-dictionary-with-list-comprehension-in-python

fao_cat_dict = collections.defaultdict(set) # ... dict(list) ... [k].append ...
#fao_cat_dict = {fao_cat_dict[key] for (key, value) in fao_cat_tuples}
for (k, v) in fao_cat_tuples:
    fao_cat_dict[k].add(v)
fao_cat_dict

defaultdict(set,
            {0: {'Oilcrops, Other'},
             1: {'Pulses, Other and products'},
             2: {'Pepper'},
             3: {'Cereals - Excluding Beer'},
             4: {'Cassava and products'},
             5: {'Dates'},
             6: {'Cephalopods'},
             7: {'Maize Germ Oil'},
             8: {'Nuts and products'},
             9: {'Cereals, Other'},
             10: {'Bananas'},
             11: {'Cocoa Beans and products'},
             12: {'Poultry Meat'},
             13: {'Mutton & Goat Meat'},
             14: {'Pigmeat'},
             15: {'Cottonseed'},
             16: {'Bovine Meat'},
             17: {'Stimulants'},
             18: {'Offals'},
             19: {'Pineapples and products'},
             20: {'Fish, Body Oil'},
             21: {'Cream'},
             22: {'Rice (Milled Equivalent)'},
             23: {'Aquatic Products, Other'},
             24: {'Crustaceans'},
             25: {'Offals, Edible'},
             26: {'Cotto

In [36]:
#print formatted for easy read
for k,v in fao_cat_dict.items():
    print(k)
    print('--------------------------')
    print(v)
    print('--------------------------')

0
--------------------------
{'Oilcrops, Other'}
--------------------------
1
--------------------------
{'Pulses, Other and products'}
--------------------------
2
--------------------------
{'Pepper'}
--------------------------
3
--------------------------
{'Cereals - Excluding Beer'}
--------------------------
4
--------------------------
{'Cassava and products'}
--------------------------
5
--------------------------
{'Dates'}
--------------------------
6
--------------------------
{'Cephalopods'}
--------------------------
7
--------------------------
{'Maize Germ Oil'}
--------------------------
8
--------------------------
{'Nuts and products'}
--------------------------
9
--------------------------
{'Cereals, Other'}
--------------------------
10
--------------------------
{'Bananas'}
--------------------------
11
--------------------------
{'Cocoa Beans and products'}
--------------------------
12
--------------------------
{'Poultry Meat'}
--------------------------
13
------

In [37]:
#print formatted for easy copy/paste
for k,v in fao_cat_dict.items():
    print(f'{k} = {v}')

0 = {'Oilcrops, Other'}
1 = {'Pulses, Other and products'}
2 = {'Pepper'}
3 = {'Cereals - Excluding Beer'}
4 = {'Cassava and products'}
5 = {'Dates'}
6 = {'Cephalopods'}
7 = {'Maize Germ Oil'}
8 = {'Nuts and products'}
9 = {'Cereals, Other'}
10 = {'Bananas'}
11 = {'Cocoa Beans and products'}
12 = {'Poultry Meat'}
13 = {'Mutton & Goat Meat'}
14 = {'Pigmeat'}
15 = {'Cottonseed'}
16 = {'Bovine Meat'}
17 = {'Stimulants'}
18 = {'Offals'}
19 = {'Pineapples and products'}
20 = {'Fish, Body Oil'}
21 = {'Cream'}
22 = {'Rice (Milled Equivalent)'}
23 = {'Aquatic Products, Other'}
24 = {'Crustaceans'}
25 = {'Offals, Edible'}
26 = {'Cottonseed Oil'}
27 = {'Coconuts - Incl Copra'}
28 = {'Ricebran Oil'}
29 = {'Freshwater Fish'}
30 = {'Butter, Ghee'}
31 = {'Meat, Other'}
32 = {'Grapes and products (excl wine)'}
33 = {'Sugar cane'}
34 = {'Eggs'}
35 = {'Infant food'}
36 = {'Roots, Other'}
37 = {'Animal fats'}
38 = {'Demersal Fish'}
39 = {'Treenuts'}
40 = {'Oilcrops'}
41 = {'Rape and Mustardseed'}
42 = {

In [38]:
# OFD with ingredients for potential categorizations ordered by column with fewest NaN
food_ing_df = ofd_data_df[ofd_data_df.columns.difference(['states', 'states_tags', 'states_en','additives', 'additives_en', 'additives_n' ])]
food_ing_df = food_ing_df[['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime', 'code',
       'countries_en', 'countries_tags', 'countries', 'product_name', 'brands',
       'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 'manufacturing_places_tags',
       'allergens']].copy()
#food_ing_df = food_df.dropna(axis=0, subset=['categories_en'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['energy_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['proteins_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['carbohydrates_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['saturated-fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fiber_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['fat_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['sugars_100g'])
food_ing_df = food_ing_df.dropna(axis=0, subset=['ingredients_text'])
food_ing_df.columns

Index(['ingredients_text', 'created_t', 'url', 'creator', 'created_datetime',
       'code', 'countries_en', 'countries_tags', 'countries', 'product_name',
       'brands', 'brands_tags', 'energy_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_from_palm_oil_n', 'fat_100g', 'sugars_100g',
       'carbohydrates_100g', 'saturated-fat_100g', 'nutrition-score-fr_100g',
       'nutrition_grade_fr', 'nutrition-score-uk_100g', 'fiber_100g',
       'serving_size', 'additives_tags', 'cholesterol_100g', 'trans-fat_100g',
       'calcium_100g', 'vitamin-c_100g', 'iron_100g', 'vitamin-a_100g',
       'pnns_groups_2', 'pnns_groups_1', 'quantity', 'categories_tags',
       'main_category_en', 'main_category', 'categories', 'categories_en',
       'packaging_tags', 'packaging', 'image_url', 'image_small_url',
       'purchase_places', 'labels_en', 'labels', 'labels_tags', 'generic_name',
       'stores', 'manufacturing_places', 

In [39]:
# create dataframe from ingredients list and pare down to main ingredient (first) and remove non-english characters using regex
# write pared ingredient list to csv
OFD_ing_df = pd.DataFrame({'OFD_Ingredients': list(set(food_ing_df['ingredients_text']))})
#OFD_ctgry_df['en_boolean'] = OFD_ctgry_df['OFD_Categories'].apply(lambda x: 'True' if re.search(r'en:', x) == True else "False")
OFD_ing_df['Main_Ingredient'] = OFD_ing_df['OFD_Ingredients'].str.split(',').str[0]
OFD_ing_df = OFD_ing_df[~OFD_ing_df['Main_Ingredient'].astype(str).str.contains(r'[^A-Za-z]', regex=True)]
#OFD_ing_df
OFD_ing_df['Main_Ingredient'].to_csv('./../OFD_ing_df.csv')
OFD_Ing_List = list(OFD_ing_df['Main_Ingredient'])

In [40]:
# get length of ingredient list
len(OFD_Ing_List)

37355

In [42]:
# dictionary from list of lists adds an unwanted layer to dictionary

Oil = ['Oilcrops', 'Oil']
Grain = ['Pulse', 'Cereal', 'Rice', 'Barley', 'Oat', 'Maize', 'Bean', 'Rye', 'Millet', 'Sorghum', 'Wheat', 'Lentil', 'Lentil']
Spice = ['Pepper', 'Cloves', 'Spice']
Plant = ['Cassava', 'Root', 'Plant', 'Yam', 'Plantain',  'potatoes', 'potato', 'Onions']
Fruit = ['Date', 'Banana', 'Pineapple', 'Grape', 'Lemon', 'Lime', 'Fruits', 'Apples', 'Olives', 'Fruit', 'Grapefruit', 'Orange', 'Mandarin', 'Citrus']
Seafood = ['Cephalopod', 'Squid', 'Cuttlefish', 'Octopus', 'Aquatic', 'Crustacean', 'Crab', 'Shrimp', 'Langoustine', 'Lobster', 'Sea', 'Fish', 'Pelagic', 'Molluscs', 'Clam', 'Oyster', 'Mussel', 'Marine', 'Seafood', 'Shellfish', 'Freshwater']
Seed_Nut= ['Seed', 'Nut', 'Cottonseed', 'Rapeseed', 'Mustardseed', 'Sunflower', 'Sesame', 'Groundnut', 'Peanut', 'Treenut', 'Almond', 'Walnut', 'Pecan', 'Cashew', 'Macadamia', 'Brazil']
Alcohol = ['Alcohol', 'Alcoholic', 'Beer', 'Wine', 'Vodka', 'Whiskey', 'Rum', 'Cider']
Dry_Goods = ['Cocoa', 'Coffee', 'Tea', 'Mate']
Meat = ['Poultry', 'Chicken', 'Turkey', 'Mutton', 'Goat', 'Lamb', 'Pigmeat', 'Pig', 'Pork', 'Bovine', 'Cow', 'Beef', 'Offal', 'Meat', 'Egg', 'Animal', 'Fat']
Dairy = ['Cream', 'Butter', 'Ghee', 'Milk', 'Butter']
Sweet = ['Cane', 'Honey', 'Sugar', 'Sweet', 'Sweeteners', 'Candy', 'Choco']
Infant = ['Infant food']
Other = ['Miscellaneous']
Vegetable = ['Pimento', 'Vegetables', 'Tomatoes', 'Pea', 'Vegetable', 'Cabbage', 'Broccoli', 'Cauliflower', 'Kale', 'Sprout', 'Cruciferous', 'Cucumber']
Palm = ['Palm']
Soy = ['Soy']
Coconut = ['Coconut', 'Copra']
list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']

map_list = [Oil, Grain, Spice, Plant, Fruit, Seafood, Seed_Nut, Alcohol, Dry_Goods, Meat, Dairy, Sweet, Infant, Other, Vegetable, Palm, Soy, Coconut]


cat_zip = list(zip(list_cats, map_list))
cat_zip
map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict
#print(list(zip(Oil)))
#print(list(zip(map_list)))
#cat_map = {[Oil, Grain]}

{'Oil': [['Oilcrops', 'Oil']],
 'Grain': [['Pulse',
   'Cereal',
   'Rice',
   'Barley',
   'Oat',
   'Maize',
   'Bean',
   'Rye',
   'Millet',
   'Sorghum',
   'Wheat',
   'Lentil',
   'Lentil']],
 'Spice': [['Pepper', 'Cloves', 'Spice']],
 'Plant': [['Cassava',
   'Root',
   'Plant',
   'Yam',
   'Plantain',
   'potatoes',
   'potato',
   'Onions']],
 'Fruit': [['Date',
   'Banana',
   'Pineapple',
   'Grape',
   'Lemon',
   'Lime',
   'Fruits',
   'Apples',
   'Olives',
   'Fruit',
   'Grapefruit',
   'Orange',
   'Mandarin',
   'Citrus']],
 'Seafood': [['Cephalopod',
   'Squid',
   'Cuttlefish',
   'Octopus',
   'Aquatic',
   'Crustacean',
   'Crab',
   'Shrimp',
   'Langoustine',
   'Lobster',
   'Sea',
   'Fish',
   'Pelagic',
   'Molluscs',
   'Clam',
   'Oyster',
   'Mussel',
   'Marine',
   'Seafood',
   'Shellfish',
   'Freshwater']],
 'Seed_Nut': [['Seed',
   'Nut',
   'Cottonseed',
   'Rapeseed',
   'Mustardseed',
   'Sunflower',
   'Sesame',
   'Groundnut',
   'Peanut',
 

In [43]:
# dictionary from manually formatted list of strings feeds into list(zip()) correctly

Oil_String = 'Oilcrops, Oil'
Grain_String = 'Pulse, Cereal, Rice, Barley, Oat, Maize, Bean, Rye, Millet, Sorghum, Wheat, Lentil, Lentil' 
Spice_String = 'Pepper, Cloves, Spice'
Plant_String = 'Cassava, Root, Plant, Yam, Plantain,  potatoes, potato, Onions'
Fruit_String = 'Date, Banana, Pineapple, Grape, Lemon, Lime, Fruits, Apples, Olives, Fruit, Grapefruit, Orange, Mandarin, Citrus' 
Seafood_String = 'Cephalopod, Squid, Cuttlefish, Octopus, Aquatic, Crustacean, Crab, Shrimp, Langoustine, Lobster, Sea, Fish, Pelagic, Molluscs, Clam, Oyster, Mussel, Marine, Seafood, Shellfish, Freshwater'
Seed_Nut_String = 'Seed, Nut, Cottonseed, Rapeseed, Mustardseed, Sunflower, Sesame, Groundnut, Peanut, Treenut, Almond, Walnut, Pecan, Cashew, Macadamia, Brazil' 
Alcohol_String = 'Alcohol, Alcoholic, Beer, Wine, Vodka, Whiskey, Rum, Cider'
Dry_Goods_String = 'Cocoa, Coffee, Tea, Mate'
Meat_String = 'Poultry, Chicken, Turkey, Mutton, Goat, Lamb, Pigmeat, Pig, Pork, Bovine, Cow, Beef, Offal, Meat, Egg, Animal, Fat' 
Dairy_String = 'Cream, Butter, Ghee, Milk, Butter'
Sweet_String = 'Cane, Honey, Sugar, Sweet, Sweeteners, Candy, Choco'
Infant_String = 'Infant food'
Other_String = 'Miscellaneous'
Vegetable_String = 'Pimento, Vegetables, Tomatoes, Pea, Vegetable, Cabbage, Broccoli, Cauliflower, Kale, Sprout, Cruciferous, Cucumber'
Palm_String = 'Palm'
Soy_String = 'Soy'
Coconut_String = 'Coconut, Copra'

string_list = [Oil_String, Grain_String, Spice_String, Plant_String, Fruit_String, Seafood_String, Seed_Nut_String, Alcohol_String, Dry_Goods_String, Meat_String, Dairy_String, Sweet_String, Infant_String, Other_String, Vegetable_String, Palm_String, Soy_String, Coconut]

list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']

cat_zip = list(zip(list_cats, string_list))
cat_zip
map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict

{'Oil': ['Oilcrops, Oil'],
 'Grain': ['Pulse, Cereal, Rice, Barley, Oat, Maize, Bean, Rye, Millet, Sorghum, Wheat, Lentil, Lentil'],
 'Spice': ['Pepper, Cloves, Spice'],
 'Plant': ['Cassava, Root, Plant, Yam, Plantain,  potatoes, potato, Onions'],
 'Fruit': ['Date, Banana, Pineapple, Grape, Lemon, Lime, Fruits, Apples, Olives, Fruit, Grapefruit, Orange, Mandarin, Citrus'],
 'Seafood': ['Cephalopod, Squid, Cuttlefish, Octopus, Aquatic, Crustacean, Crab, Shrimp, Langoustine, Lobster, Sea, Fish, Pelagic, Molluscs, Clam, Oyster, Mussel, Marine, Seafood, Shellfish, Freshwater'],
 'Seed_Nut': ['Seed, Nut, Cottonseed, Rapeseed, Mustardseed, Sunflower, Sesame, Groundnut, Peanut, Treenut, Almond, Walnut, Pecan, Cashew, Macadamia, Brazil'],
 'Alcohol': ['Alcohol, Alcoholic, Beer, Wine, Vodka, Whiskey, Rum, Cider'],
 'Dry_Goods': ['Cocoa, Coffee, Tea, Mate'],
 'Meat': ['Poultry, Chicken, Turkey, Mutton, Goat, Lamb, Pigmeat, Pig, Pork, Bovine, Cow, Beef, Offal, Meat, Egg, Animal, Fat'],
 'Dairy': 

In [44]:
# flatten list of list of strings into new list of strings to try a different method of creating a dictionary because of access issues
Oil = ['Oilcrops', 'Oil']
Grain = ['Pulse', 'Cereal', 'Rice', 'Barley', 'Oat', 'Maize', 'Bean', 'Rye', 'Millet', 'Sorghum', 'Wheat', 'Lentil', 'Lentil']
Spice = ['Pepper', 'Cloves', 'Spice']
Plant = ['Cassava', 'Root', 'Plant', 'Yam', 'Plantain',  'potatoes', 'potato', 'Onions']
Fruit = ['Date', 'Banana', 'Pineapple', 'Grape', 'Lemon', 'Lime', 'Fruits', 'Apples', 'Olives', 'Fruit', 'Grapefruit', 'Orange', 'Mandarin', 'Citrus']
Seafood = ['Cephalopod', 'Squid', 'Cuttlefish', 'Octopus', 'Aquatic', 'Crustacean', 'Crab', 'Shrimp', 'Langoustine', 'Lobster', 'Sea', 'Fish', 'Pelagic', 'Molluscs', 'Clam', 'Oyster', 'Mussel', 'Marine', 'Seafood', 'Shellfish', 'Freshwater']
Seed_Nut= ['Seed', 'Nut', 'Cottonseed', 'Rapeseed', 'Mustardseed', 'Sunflower', 'Sesame', 'Groundnut', 'Peanut', 'Treenut', 'Almond', 'Walnut', 'Pecan', 'Cashew', 'Macadamia', 'Brazil']
Alcohol = ['Alcohol', 'Alcoholic', 'Beer', 'Wine', 'Vodka', 'Whiskey', 'Rum', 'Cider']
Dry_Goods = ['Cocoa', 'Coffee', 'Tea', 'Mate']
Meat = ['Poultry', 'Chicken', 'Turkey', 'Mutton', 'Goat', 'Lamb', 'Pigmeat', 'Pig', 'Pork', 'Bovine', 'Cow', 'Beef', 'Offal', 'Meat', 'Egg', 'Animal', 'Fat']
Dairy = ['Cream', 'Butter', 'Ghee', 'Milk', 'Butter']
Sweet = ['Cane', 'Honey', 'Sugar', 'Sweet', 'Sweeteners', 'Candy', 'Choco']
Infant = ['Infant food']
Other = ['Miscellaneous']
Vegetable = ['Pimento', 'Vegetables', 'Tomatoes', 'Pea', 'Vegetable', 'Cabbage', 'Broccoli', 'Cauliflower', 'Kale', 'Sprout', 'Cruciferous', 'Cucumber']
Palm = ['Palm']
Soy = ['Soy']
Coconut = ['Coconut', 'Copra']
list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']

map_list = [Oil, Grain, Spice, Plant, Fruit, Seafood, Seed_Nut, Alcohol, Dry_Goods, Meat, Dairy, Sweet, Infant, Other, Vegetable, Palm, Soy, Coconut]

flat_list = []
for item in map_list:
    item = re.sub(r"'", '', str(item)).strip('[]')
    #item = ''.join(('\'', item, '\''))
    #item = item.strip('[]')
    #item = tuple(item)
    #str(item)
    #print(item)
    flat_list.append(item)

cat_zip = list(zip(list_cats, flat_list))

map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict

{'Oil': ['Oilcrops, Oil'],
 'Grain': ['Pulse, Cereal, Rice, Barley, Oat, Maize, Bean, Rye, Millet, Sorghum, Wheat, Lentil, Lentil'],
 'Spice': ['Pepper, Cloves, Spice'],
 'Plant': ['Cassava, Root, Plant, Yam, Plantain, potatoes, potato, Onions'],
 'Fruit': ['Date, Banana, Pineapple, Grape, Lemon, Lime, Fruits, Apples, Olives, Fruit, Grapefruit, Orange, Mandarin, Citrus'],
 'Seafood': ['Cephalopod, Squid, Cuttlefish, Octopus, Aquatic, Crustacean, Crab, Shrimp, Langoustine, Lobster, Sea, Fish, Pelagic, Molluscs, Clam, Oyster, Mussel, Marine, Seafood, Shellfish, Freshwater'],
 'Seed_Nut': ['Seed, Nut, Cottonseed, Rapeseed, Mustardseed, Sunflower, Sesame, Groundnut, Peanut, Treenut, Almond, Walnut, Pecan, Cashew, Macadamia, Brazil'],
 'Alcohol': ['Alcohol, Alcoholic, Beer, Wine, Vodka, Whiskey, Rum, Cider'],
 'Dry_Goods': ['Cocoa, Coffee, Tea, Mate'],
 'Meat': ['Poultry, Chicken, Turkey, Mutton, Goat, Lamb, Pigmeat, Pig, Pork, Bovine, Cow, Beef, Offal, Meat, Egg, Animal, Fat'],
 'Dairy': [

In [45]:
# yet another attempt at creating the dictionary differenty 
# flatten list of list of strings into new list of strings

Oil = (('Oilcrops'), ('Oil'))
Grain = (('Pulse'), ('Cereal'), ('Rice'), ('Barley'), ('Oat'), ('Maize'), ('Bean'), ('Rye'), ('Millet'), ('Sorghum'), ('Wheat'), ('Lentil'), ('Lentil'))
Spice = (('Pepper'), ('Cloves'), ('Spice'))
Plant = (('Cassava'), ('Root'), ('Plant'), ('Yam'), ('Plantain'),  ('potatoes'), ('potato'), ('Onions'))
Fruit = (('Date'), ('Banana'), ('Pineapple'), ('Grape'), ('Lemon'), ('Lime'), ('Fruits'), ('Apples'), ('Olives'), ('Fruit'), ('Grapefruit'), ('Orange'), ('Mandarin'), ('Citrus'))
Seafood = (('Cephalopod'), ('Squid'), ('Cuttlefish'), ('Octopus'), ('Aquatic'), ('Crustacean'), ('Crab'), ('Shrimp'), ('Langoustine'), ('Lobster'), ('Sea'), ('Fish'), ('Pelagic'), ('Molluscs'), ('Clam'), ('Oyster'), ('Mussel'), ('Marine'), ('Seafood'), ('Shellfish'), ('Freshwater'))
Seed_Nut= (('Seed'), ('Nut'), ('Cottonseed'), ('Rapeseed'), ('Mustardseed'), ('Sunflower'), ('Sesame'), ('Groundnut'), ('Peanut'), ('Treenut'), ('Almond'), ('Walnut'), ('Pecan'), ('Cashew'), ('Macadamia'), ('Brazil'))
Alcohol = (('Alcohol'), ('Alcoholic'), ('Beer'), ('Wine'), ('Vodka'), ('Whiskey'), ('Rum'), ('Cider'))
Dry_Goods = (('Cocoa'), ('Coffee'), ('Tea'), ('Mate'))
Meat = (('Poultry'), ('Chicken'), ('Turkey'), ('Mutton'), ('Goat'), ('Lamb'), ('Pigmeat'), ('Pig'), ('Pork'), ('Bovine'), ('Cow'), ('Beef'), ('Offal'), ('Meat'), ('Egg'), ('Animal'), ('Fat'))
Dairy = (('Cream'), ('Butter'), ('Ghee'), ('Milk'), ('Butter'))
Sweet = (('Cane'), ('Honey'), ('Sugar'), ('Sweet'), ('Sweeteners'), ('Candy'), ('Choco'))
Infant = (('Infant food'))
Other = (('Miscellaneous'))
Vegetable = (('Pimento'), ('Vegetables'), ('Tomatoes'), ('Pea'), ('Vegetable'), ('Cabbage'), ('Broccoli'), ('Cauliflower'), ('Kale'), ('Sprout'), ('Cruciferous'), ('Cucumber'))
Palm = (('Palm'))
Soy = (('Soy'))
Coconut = (('Coconut'), ('Copra'))


list_cats = ['Oil', 'Grain', 'Spice', 'Plant', 'Fruit', 'Seafood', 'Seed_Nut', 'Alcohol', 'Dry_Goods', 'Meat', 'Dairy', 'Sweet', 'Infant', 'Other', 'Vegetable', 'Palm', 'Soy', 'Coconut']


map_list = [Oil, Grain, Spice, Plant, Fruit, Seafood, Seed_Nut, Alcohol, Dry_Goods, Meat, Dairy, Sweet, Infant, Other, Vegetable, Palm, Soy, Coconut]

cat_zip = list(zip(list_cats, map_list))

map_dict = {}
for (key, value) in cat_zip:
    if key not in map_dict:
        map_dict[key] = []
    map_dict[key].append(value)
map_dict

{'Oil': [('Oilcrops', 'Oil')],
 'Grain': [('Pulse',
   'Cereal',
   'Rice',
   'Barley',
   'Oat',
   'Maize',
   'Bean',
   'Rye',
   'Millet',
   'Sorghum',
   'Wheat',
   'Lentil',
   'Lentil')],
 'Spice': [('Pepper', 'Cloves', 'Spice')],
 'Plant': [('Cassava',
   'Root',
   'Plant',
   'Yam',
   'Plantain',
   'potatoes',
   'potato',
   'Onions')],
 'Fruit': [('Date',
   'Banana',
   'Pineapple',
   'Grape',
   'Lemon',
   'Lime',
   'Fruits',
   'Apples',
   'Olives',
   'Fruit',
   'Grapefruit',
   'Orange',
   'Mandarin',
   'Citrus')],
 'Seafood': [('Cephalopod',
   'Squid',
   'Cuttlefish',
   'Octopus',
   'Aquatic',
   'Crustacean',
   'Crab',
   'Shrimp',
   'Langoustine',
   'Lobster',
   'Sea',
   'Fish',
   'Pelagic',
   'Molluscs',
   'Clam',
   'Oyster',
   'Mussel',
   'Marine',
   'Seafood',
   'Shellfish',
   'Freshwater')],
 'Seed_Nut': [('Seed',
   'Nut',
   'Cottonseed',
   'Rapeseed',
   'Mustardseed',
   'Sunflower',
   'Sesame',
   'Groundnut',
   'Peanut',
 

In [46]:
# Effort to loop through ingredients list
for k, v in map_dict.items():
    for i in OFD_Ing_List:
        if i in k:
            print(i)
            print(f'--------------------------------------------\n' 
                  f'{i} IS in dict in {k}, {v}\n'
                  f'--------------------------------------------\n')
        #else:
        #    print(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n{i} is NOT in dict\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n')

Oil
--------------------------------------------
Oil IS in dict in Oil, [('Oilcrops', 'Oil')]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [('Oilcrops', 'Oil')]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [('Oilcrops', 'Oil')]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [('Oilcrops', 'Oil')]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [('Oilcrops', 'Oil')]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [('Oilcrops', 'Oil')]
--------------------------------------------


--------------------------------------------
 IS in dict in Oil, [('Oilcrops', 'Oil')]
--------------------------------------------


--------------------------------------------
 IS in dic

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [('Coconut', 'Copra')]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [('Coconut', 'Copra')]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [('Coconut', 'Copra')]
--------------------------------------------


--------------------------------------------
 IS in dict in Coconut, [('Coconut', 'Copra')]
--------------------------------------------

Coconut
--------------------------------------------
Coconut IS in dict in Coconut, [('Coconut', 'Copra')]
--------------------------------------------


--------------------------------------------
 IS in dict in Coconut, [('Coconut', 'Copra')]
--------------------------------------------

o
--------------------------------------------
o IS in dict in Coconut, [('Coconut', 'Copra')]
---------------

In [47]:
d = {'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)}


In [48]:
# more looping effor with sample dict
for k, v in d.items():
    if 'one' in v:
        print('one')

one


C:\Users\soria\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  app.launch_new_instance()


In [49]:
#define function to comb through the other category list
def food(iterable):
    for k, v in map_dict.items():
        if iterable in v:
            print(f'{iterable} is in a category\n----------------------------')
        else:
            print(f'{iterable} is NOT in a category\n-------------------------')
#food(OFD_Ing_List)      

In [50]:
# trying to use magic % operator and globals to dynamically create list names

# https://stackoverflow.com/questions/6181935/how-do-you-create-different-variable-names-while-in-a-loop
# https://developers.google.com/edu/python/dict-files
#for k, v in cat_dict.items():
    #globals()['k%s' % k] = v
    #%(k) = %(v)
    

#for k in cat_dict.keys():
    #globals()['k%s' % k] = 'me'